In [1]:
import torch
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import torch.nn.functional as F
import math

In [ ]:
model_utils = {
    "resenet50": "https://download.pytorch.org/models/resnet50-19c8e357.pth"
}

def conv3x3(in_planes, out_planes, stride=1):
    """3*3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class Bottleneck(nn.Module):
    # Resnet-B
    expansion = 4
    
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.
        

# 检测分支

In [ ]:
class ClsCntRegHead(nn.Module):
    """检测分支

    Args:
        nn (_type_): _description_
    """
    def __init__(self, in_channel, class_num, GN=True, cnt_on_reg=True, prior=0.01):
        super(ClsCntRegHead, self).__init__()
        self.prior = prior
        self.class_num = class_num
        self.cnt_on_reg = cnt_on_reg
        
        # 1 ========================从fpn到head的侧连===========================
        cls_branch = []
        reg_branch = []
        for i in range(4):
            # cls_branch: conv--gn--relu  这里的卷积都不改变图像尺寸
            cls_branch.append(nn.Conv2d(in_channel, in_channel, kernel_size=3, padding=1, bias = True))
            if GN:
                reg_branch.append(nn.GroupNorm(32, in_channel))
            cls_branch.append(nn.ReLU(True))
            
            # reg_branch: conv--gn--relu  这里的卷积都不改变图像尺寸
            reg_branch.append(nn.Conv2d(in_channel, in_channel, kernel_size=3, padding=1, bias = True))
            if GN:
                reg_branch.append(nn.GroupNorm(32, in_channel))
            reg_branch.append(nn.ReLU(True))
        
        # 1.1 分类网络分支
        self.cls_conv = nn.Sequential(*cls_branch)  # 将cls_branch中的参数，一个个拆解成独立的参数；列表或者元组之前加一个*，字典前加两个**
        # 1.2 回归网络分支
        self.reg_conv = nn.Sequential(*reg_branch)
        print(' Bypasses of the detection head;')
        print(self.cls_conv, '\n')
        print(self.reg_conv)
        # =======================================================================
        
        # 2 ========================网络输出=====================================
        # 2.1 网络分类路径输出
        
        
        
        
        